In [3]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=8141151789fed36700c7d5c8d7fbc81671fbf30e4eb4c308d083019e3e97a942
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [16]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt


In [18]:
df = pd.read_csv('/content/drive/MyDrive/CEG/SEM 2/Big Data/Mini Project BD/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']


In [19]:
# Remove unnecessary columns
df = df[['target', 'text']]

# Preprocess the text
# Add your text preprocessing steps here using scikit-learn or other libraries

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)

# Apply TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Convert labels to numeric values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Train the classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_tfidf, y_train_encoded)

In [ ]:
# Make predictions
y_pred = clf.predict(X_test_tfidf)
df1 = pd.DataFrame({'Actual': y_test_encoded, 'Predicted': y_pred})
df1

In [ ]:

# Evaluate the model
accuracy = accuracy_score(y_test_encoded, y_pred)
confusion_mat = confusion_matrix(y_test_encoded, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat)

In [ ]:
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()
spark_df = spark.createDataFrame(pd.DataFrame(X_train_tfidf.toarray()), ["features"])
spark_df.show()


In [ ]:
sentiment_counts = pd.Series(label_encoder.inverse_transform(y_train_encoded)).value_counts()
sentiment_counts.plot(kind='bar')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Sentiment Distribution')
plt.show()
